# Fitting a hierarchical logistic model to German credit data

This notebook explains how to run the toy hierarchical logistic regression model example using the German credit data from [1]. In this example, we have predictors for 1000 individuals and an outcome variable indicating whether or not each individual should be given credit.


[1] "UCI machine learning repository", 2010. A. Frank and A. Asuncion. https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)

In [1]:
import pints
import pints.toy
import pints.plot
import numpy as np
import matplotlib.pyplot as plt
import io
import urllib
from scipy import stats
import time

To run this example, we need to first get the data from [1] and process it so we have dichtonomous $y\in\{-1,1\}$ outputs and the matrix of predictors has been standardised. In addition, we also add a column of 1s corresponding to a constant term in the regression.

If you are connected to the internet, by instantiating with `x=None`, Pints will fetch the data from the repo for you. If, instead, you have local copies of the `x` and `y` matrices, these can be supplied as arguments. 

In [2]:
logpdf = pints.toy.GermanCreditHierarchicalLogPDF(download=True)

Let's look at the data: `x` is a matrix of predictors and `y` is a vector of credit recommendations for 1000 individuals. Pints also handles processing of `x` into a design matrix `z` of all interactions between variables (including with themselves).

In [3]:
x, y, z = logpdf.data()
print(z.shape)

(1000, 325)


Now we run HMC to fit the parameters of the model.

In [4]:
xs = [
    np.random.uniform(0, 1, size=(logpdf.n_parameters())),
    np.random.uniform(0, 1, size=(logpdf.n_parameters())),
    np.random.uniform(0, 1, size=(logpdf.n_parameters())),
]

mcmc = pints.MCMCController(logpdf, len(xs), xs, method=pints.HamiltonianMCMC)
mcmc.set_max_iterations(400)

# Set up modest logging
mcmc.set_log_to_screen(True)
mcmc.set_log_interval(10)

for sampler in mcmc.samplers():
    sampler.set_leapfrog_step_size(0.02)
    sampler.set_leapfrog_steps(1)

start = time.time()    

# Run!
print('Running...')
chains = mcmc.run()
print('Done!')

end = time.time()  
diff = end - start

Running...
Using Hamiltonian Monte Carlo
Generating 3 chains.
Running in sequential mode.
Iter. Eval. Accept.   Accept.   Accept.   Time m:s
0     3      0         0         0          0:00.3
1     6      0         0.333     0          0:00.9
2     9      0         0.5       0          0:01.7
3     12     0.2       0.6       0.2        0:02.3
10    33     0.333     0.833     0.0833     0:06.0
20    63     0.5       0.818     0.136      0:11.1
30    93     0.46875   0.8125    0.1875     0:16.5
40    123    0.476     0.786     0.238      0:21.3
50    153    0.404     0.788     0.269      0:26.1
60    183    0.371     0.774     0.306      0:30.5
70    213    0.319     0.792     0.292      0:35.5
80    243    0.293     0.804878  0.317      0:40.0
90    273    0.272     0.804     0.337      0:44.8
100   303    0.284     0.775     0.343      0:47.9
110   333    0.286     0.786     0.33       0:50.4
120   363    0.279     0.779     0.32       0:52.8
130   393    0.295     0.758     0.318     

This is clearly a much harder problem than the non-hierarchical version!

In [5]:
results = pints.MCMCSummary(chains=chains, time=diff)
print(results)

param      mean    std.    2.5%    25%    50%    75%    97.5%    rhat    ess    ess per sec.
---------  ------  ------  ------  -----  -----  -----  -------  ------  -----  --------------
param 1    0.63    0.12    0.51    0.54   0.59   0.67   0.95     3.50    8.76   0.08
param 2    0.24    0.32    -0.03   0.01   0.05   0.63   0.81     14.55   8.80   0.08
param 3    0.16    0.09    0.03    0.05   0.16   0.26   0.28     19.37   10.48  0.09
param 4    0.39    0.37    0.09    0.10   0.16   0.90   0.92     54.18   13.35  0.11
param 5    0.42    0.33    -0.00   0.03   0.42   0.83   0.84     39.32   16.06  0.14
param 6    0.57    0.26    0.17    0.21   0.71   0.79   0.80     42.75   15.82  0.14
param 7    0.41    0.32    0.06    0.08   0.30   0.83   0.88     43.58   14.72  0.13
param 8    0.37    0.22    0.11    0.12   0.34   0.65   0.67     35.26   22.56  0.19
param 9    0.32    0.08    0.19    0.21   0.36   0.39   0.41     11.90   18.41  0.16
param 10   0.47    0.11    0.33    0.35   0.45 